In [1]:
import pandas as pd
import pickle 
import json
import TransformHelper

In [2]:
def save_to_disk(data, filepath):
    with open(filepath, 'wb') as file:
        pickle.dump(data, file)
def load_from_disk(filepath):
    with open(filepath, 'rb') as file:
        data = pickle.load(file)
        return data
def read_normalized_json_to_df(filepath):
    full_file_df = ""
    with open(filepath, 'r', encoding="utf-8") as json_file:
        json_full = json.load(json_file)
    full_file_df = pd.json_normalize(json_full)
    return full_file_df

In [4]:
json_path = "./AllBuildingList_TaskAssignment.json"

## Convention

Building names list contains 
* Actual buildings ("Building_abc")
* Task buildings (e.g. "TaskBuilding_ab")
* Pre-defined Landmarks (e.g. "Castle-TaskBuilding_56")
* Cranes (e.g. crane_a)
* Garages (e.g. "Garage_abc")

Actual buildings: Are filtered such that their name appears only once in the building list. Some buildings have multiple colliders (connected to the same building name), use only one of those however for plotting purposes. 
Task buildings: Names are included as they are in the list. 
Pre-defined landmarks: Names are included as they are in the list. 
Cranes: Colliders are filtered out from measurement data and added to the list.
Garages: Some garages are treated as seperate buildinngs, some are considered as part of the building they are close to. Specifically, Garage_235, Garage_224, Garage_185, Garage_98, Garage_86 are seperate buildings and treated as such; Garage_72, Garage_71, Garage_46, Garage_26 are treated as belonging to the buildings with the same numbers, i.e. those garages are renamed (e.g. Garage_72 becomes Building_72). 


### Building name list

In [3]:
# Read Json 
with open(json_path, 'r', encoding="utf-8") as json_file:
    buildings_dict = json.load(json_file)

In [4]:
# Extract building names 
all_building_names = []
for elem in buildings_dict:
    all_building_names.append(elem["BuildingName"])
print("All Building names (" + str(len(all_building_names)) + ", unique names " + str(len(list(set((all_building_names))))) + "): \n", all_building_names)

All Building names (284, unique names 247): 
 ['Building_92', 'Garage_224', 'Building_226', 'Building_58', 'Building_182', 'Building_101', 'Building_225', 'Building_105', 'Building_139', 'Building_161', 'Garage_185', 'Building_146', 'Building_229', 'Building_86', 'Building_216', 'Building_215', 'Building_175', 'Building_196', 'Building_228', 'Building_124', 'Building_115', 'Building_192', 'Building_174', 'Building_219', 'Building_213', 'Building_192', 'Building_116', 'Building_116', 'Building_115', 'Building_189', 'Building_207', 'Building_193', 'Building_158', 'Building_123', 'Building_154', 'Building_125', 'Building_206', 'Building_191', 'Building_212', 'Building_190', 'Building_151', 'Building_84', 'Building_122', 'Building_106', 'Garage_185', 'Building_198', 'Building_131', 'Building_179', 'Building_201', 'Building_165', 'Building_184', 'Building_171', 'Building_176', 'Building_177', 'Building_208', 'Building_185', 'Building_178', 'Building_121', 'Building_157', 'Building_164', 'Bu

In [5]:
# Filter buildings
exclude_parts = [] # ["garage"]
filtered_names = []
for elem in all_building_names:
    for exclude in exclude_parts:
        if exclude in elem.lower():
            filtered_names.append(elem)
filtered_names = list(set(all_building_names) - set(filtered_names))
print("Filtered names (" + str(len(filtered_names)) + "): \n", filtered_names)

Filtered names (247): 
 ['Building_154', 'Building_176', 'Building_62', 'Building_227', 'Building_141', 'Building_179', 'Building_83', 'Building_60', 'Garage_86', 'TaskBuilding_17', 'Building_125', 'Building_182', 'Building_110', 'Church-TaskBuilding_16', 'Building_126', 'Garage_46', 'Building_105', 'Garage_72', 'TaskBuilding_55', 'Building_151', 'Building_103', 'Windmill-TaskBuilding_10_1', 'Building_69', 'Building_155', 'Building_91', 'Building_123', 'Building_220', 'Building_216', 'Building_228', 'Building_131', 'Building_234', 'Building_108', 'Building_222', 'Building_130', 'TaskBuilding_38', 'Building_132', 'Building_70', 'Building_199', 'Building_211', 'TaskBuilding_31', 'Building_157', 'Building_161', 'Building_189', 'Building_194', 'Building_112', 'Building_58', 'Building_204', 'Building_162', 'TaskBuilding_47', 'TaskBuilding_26', 'Building_233', 'Building_75', 'Building_187', 'Building_120', 'Building_215', 'Building_136', 'Building_89', 'Building_165', 'Garage_224', 'Building

In [6]:
# rename buildings
rename_dict = {'Garage_235':'Building_235', 'Garage_224':'Building_224', 'Garage_185':'Building_185', \
                'Garage_71':'Building_71', 'Garage_86':'Building_86', 'Garage_98':'Building_98', \
                'Garage_26':'TaskBuilding_26', 'Garage_46':'TaskBuilding_46', 'Garage_72':'Building_72'}
renamed_buildings = []
for elem in filtered_names:
    if elem in [*rename_dict]:
        renamed_buildings.append(rename_dict[elem])
    else:
        renamed_buildings.append(elem)
print("Renamed buildings (" + str(len(renamed_buildings)) + "): \n" + str(renamed_buildings))       



Renamed buildings (247): 
['Building_154', 'Building_176', 'Building_62', 'Building_227', 'Building_141', 'Building_179', 'Building_83', 'Building_60', 'Building_86', 'TaskBuilding_17', 'Building_125', 'Building_182', 'Building_110', 'Church-TaskBuilding_16', 'Building_126', 'TaskBuilding_46', 'Building_105', 'Building_72', 'TaskBuilding_55', 'Building_151', 'Building_103', 'Windmill-TaskBuilding_10_1', 'Building_69', 'Building_155', 'Building_91', 'Building_123', 'Building_220', 'Building_216', 'Building_228', 'Building_131', 'Building_234', 'Building_108', 'Building_222', 'Building_130', 'TaskBuilding_38', 'Building_132', 'Building_70', 'Building_199', 'Building_211', 'TaskBuilding_31', 'Building_157', 'Building_161', 'Building_189', 'Building_194', 'Building_112', 'Building_58', 'Building_204', 'Building_162', 'TaskBuilding_47', 'TaskBuilding_26', 'Building_233', 'Building_75', 'Building_187', 'Building_120', 'Building_215', 'Building_136', 'Building_89', 'Building_165', 'Building_2

In [8]:
# save data
#renamed_buildings.sort()
#save_to_disk(renamed_buildings,"./building_names.pickle")

filtered_names.sort()
save_to_disk(filtered_names,"./building_names.pickle")


### Generate map coordinates for entire building json

In [10]:
# Read all buildings from the buildings json file 
nonnunique_buildings_df = read_normalized_json_to_df(json_path)

# Load transformation info 
transform_infos = load_from_disk("./map_image_transform_infos.pickle")
transform_matrix = transform_infos["perspective_transform_matrix"]
image_size = transform_infos["image_size"]

# Apply transformation 
nonnunique_buildings_df = TransformHelper.apply_perspective_transform_matrix(transform_matrix,img_size=image_size,dataframe=nonnunique_buildings_df,df_x_name="BuildingPosition.x",df_y_name="BuildingPosition.z",transform_x_name="transformed_buildingposition_x",transform_y_name="transformed_buildingposition_y")

# Save to disk (non-unique)
#save_to_disk(building_df, "./buildings_nonunique_positions.pickle")





Calculating Perspective Transform...


100%|██████████| 284/284 [00:00<00:00, 4970.40it/s]


### Find buildings with multiple different colliders

In [31]:
# Group by BuildingName 
building_cnt_df = nonnunique_buildings_df.groupby("BuildingName").count()

print("Buildings of the same name with multiple different positions (from buildings JSON):")
print(building_cnt_df.loc[building_cnt_df["BuildingID"] != 1]["BuildingID"].sort_index())
print()
print("Colliders of those buildings (from buildings JSON):")
nonnunique_buildings_df.loc[nonnunique_buildings_df["BuildingName"].isin(building_cnt_df.loc[building_cnt_df["BuildingID"] != 1].reset_index()["BuildingName"])].sort_values("BuildingName")

Buildings of the same name with multiple different positions (from buildings JSON):
BuildingName
Building_115     2
Building_116     2
Building_183     2
Building_192     2
Garage_185      16
Garage_224       4
Garage_235       8
Garage_46        3
Garage_86        4
Garage_98        4
Name: BuildingID, dtype: int64

Colliders of those buildings (from buildings JSON):


,BuildingID,BuildingName,BuildingPosition.x,BuildingPosition.y,BuildingPosition.z,BuildingRotation.x,BuildingRotation.y,BuildingRotation.z,buildingRotationQuaternion.x,buildingRotationQuaternion.y,buildingRotationQuaternion.z,buildingRotationQuaternion.w,BuildingDirection.x,BuildingDirection.y,BuildingDirection.z,transformed_buildingposition_x,transformed_buildingposition_y
20,115,Building_115,188.630005,-7.110001,-43.759998,0.000000,143.307007,0.000000,0.000000,0.949170,0.000000e+00,0.314763,5.975273e-01,0.000000e+00,-8.018489e-01,2825.868525,1865.850615
28,115,Building_115,187.184006,-7.110000,-41.734001,0.000000,323.307434,0.000000,0.000000,-0.314759,0.000000e+00,0.949172,-5.975210e-01,0.000000e+00,8.018532e-01,2819.920833,1874.196025
26,116,Building_116,173.764008,-7.110000,-51.734001,0.000000,323.307434,0.000000,0.000000,-0.314759,0.000000e+00,0.949172,-5.975210e-01,0.000000e+00,8.018532e-01,2764.750690,1833.082051
27,116,Building_116,175.210007,-7.110001,-53.759998,0.000000,143.307007,0.000000,0.000000,0.949170,0.000000e+00,0.314763,5.975273e-01,0.000000e+00,-8.018489e-01,2770.699829,1824.737129
141,183,Building_183,-183.671738,2.391739,115.689453,0.000007,47.997173,-0.000016,0.000000,0.406714,-1.490116e-07,0.913556,7.431118e-01,-1.212102e-07,6.691673e-01,1289.460134,2525.988123
183,183,Building_183,-183.671738,2.391739,115.689453,0.000007,47.997173,-0.000016,0.000000,0.406714,-1.490116e-07,0.913556,7.431118e-01,-1.212102e-07,6.691673e-01,1289.460134,2525.988123
21,192,Building_192,-386.309998,0.603148,146.440002,0.000000,136.374786,0.000000,0.000000,0.928404,0.000000e+00,0.371572,6.899382e-01,0.000000e+00,-7.238684e-01,449.342083,2655.111470
25,192,Building_192,-414.699371,6.104360,156.605362,0.000000,45.146057,0.000000,0.000000,0.383861,0.000000e+00,0.923391,7.089071e-01,0.000000e+00,7.053019e-01,331.336318,2697.503947
132,185,Garage_185,-275.526917,2.573868,155.664886,270.000000,316.191559,0.000000,-0.656060,-0.263791,-2.637905e-01,0.656060,2.980232e-08,1.000000e+00,-2.831221e-07,908.690929,2692.218566
118,185,Garage_185,-242.185562,2.834320,161.479965,270.000000,136.191589,0.000000,-0.263790,0.656060,6.560599e-01,0.263790,-2.980232e-08,1.000000e+00,-2.533197e-07,1046.766728,2715.950357


### Keep only first collider for each building name

In [34]:
# Extract unique positions (take first, and ignore others)
unique_buildings_df = nonnunique_buildings_df.drop_duplicates(subset="BuildingName",keep="first").sort_values("BuildingID")
print("Number of unique building names (from building JSON): " + str(len(unique_buildings_df)))
unique_buildings_df.head()

Number of unique building names (from building JSON): 247


,BuildingID,BuildingName,BuildingPosition.x,BuildingPosition.y,BuildingPosition.z,BuildingRotation.x,BuildingRotation.y,BuildingRotation.z,buildingRotationQuaternion.x,buildingRotationQuaternion.y,buildingRotationQuaternion.z,buildingRotationQuaternion.w,BuildingDirection.x,BuildingDirection.y,BuildingDirection.z,transformed_buildingposition_x,transformed_buildingposition_y
252,1,TaskBuilding_1,255.062286,-1.663044,-18.984482,0.000023,145.325836,-0.000005,1.490116e-08,0.954568,-2.086163e-07,0.297993,0.568909,-4.071578e-07,-0.822401,3098.884060,1967.542497
276,2,TaskBuilding_2,197.395660,-2.950989,234.249405,0.000041,171.773102,-0.000002,1.117587e-08,0.997424,-3.576279e-07,0.071732,0.143094,-7.150165e-07,-0.989709,2861.651139,3011.791157
260,3,TaskBuilding_3,132.779266,-1.083273,29.115131,0.000019,139.375046,-0.000007,0.000000e+00,0.937813,-1.788139e-07,0.347140,0.651105,-3.353882e-07,-0.758988,2595.971558,2166.281572
248,4,TaskBuilding_4,45.866539,-4.682556,-122.198311,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,1.000000,0.000000,0.000000e+00,1.000000,2238.599292,1543.316951
265,5,TaskBuilding_5,-15.720150,0.160141,34.060219,0.000005,70.209091,-0.000013,-2.980232e-08,0.575070,-1.192093e-07,0.818104,0.940934,-8.834461e-08,0.338589,1983.975134,2187.585268


### Find additional buildings from measurement data

In [44]:
measurement_df = pd.read_csv("./../Results/1023_CompleteExploration_Hitpoints.csv")
#measurement_df.loc[measurement_df["hitObjectColliderName"].str.contains("crane")].groupby("hitObjectColliderName").count()

# colliders that we want additionally in the buildings list and that appear in the measurement data
additional_colliders = ["crane_1", "crane_2"] 

measurement_df.loc[measurement_df["hitObjectColliderName"].isin(additional_colliders)].groupby(["hitObjectColliderName","hitObjectColliderBoundsCenter.x","hitObjectColliderBoundsCenter.z"]).count()


Unnamed: 0  \
hitObjectColliderName hitObjectColliderBoundsCenter.x hitObjectColliderBoundsCenter.z               
crane_1               -113.048508                     -20.889994                               19   
                      -103.404709                     -23.830784                               44   
                      -86.889000                      -28.867096                              465   
crane_2               -320.619263                     -177.796417                              13   
                      -313.990875                     -151.263092                               7   

                                                                                       timeStampDataPointStart  \
hitObjectColliderName hitObjectColliderBoundsCenter.x hitObjectColliderBoundsCenter.z                            
crane_1               -113.048508                     -20.889994                                            19   
                      -103.404709                     -23.830784                                            44   
                      -86.889000                      -28.867096                                           465   
crane_2               -320.619263                     -177.796417                                           13   
                      -313.990875                     -151.263092                                            7   

                                                                                       timeStampDataPointEnd  \
hitObjectColliderName hitObjectColliderBoundsCenter.x hitObjectColliderBoundsCenter.z                          
crane_1               -113.048508                     -20.889994                                          19   
                      -103.404709                     -23.830784                                          44   
                      -86.889000                      -28.867096                                         465   
crane_2               -320.619263                     -177.796417                                         13   
                      -313.990875                     -151.263092                                          7   

                                                                                       ordinalOfHit  \
hitObjectColliderName hitObjectColliderBoundsCenter.x hitObjectColliderBoundsCenter.z                 
crane_1               -113.048508                     -20.889994                                 19   
                      -103.404709                     -23.830784                                 44   
                      -86.889000                      -28.867096                                465   
crane_2               -320.619263                     -177.796417                                13   
                      -313.990875                     -151.263092                                 7   

                                                                                       hitPointOnObject.x  \
hitObjectColliderName hitObjectColliderBoundsCenter.x hitObjectColliderBoundsCenter.z                       
crane_1               -113.048508                     -20.889994                                       19   
                      -103.404709                     -23.830784                                       44   
                      -86.889000                      -28.867096                                      465   
crane_2               -320.619263                     -177.796417                                      13   
                      -313.990875                     -151.263092                                       7   

                                                                                       hitPointOnObject.y  \
hitObjectColliderName hitObjectColliderBoundsCenter.x hitObjectColliderBoundsCenter.z                       
crane_1               -113.048508                     -20.889994                           